In [ ]:
# GOOGLE DRIVE PDF TO EXCEL (UPDATED CLEAN TITLES + A-Z SORTED VERSION)
# Target Folder: 18iuRG9FI7d8Xn9DRh1p1gjfamZ6k0FPh

import pandas as pd
import re  # Text safayi ke liye library
from googleapiclient.discovery import build
from google.colab import auth, drive, files
from google.auth import default

# --- CONFIGURATION ---
TARGET_FOLDER_ID = "18iuRG9FI7d8Xn9DRh1p1gjfamZ6k0FPh"
OUTPUT_FILENAME = "Hashim Nadeem.xlsx"

# Step 1: Authentication
print("Authenticating...")
drive.mount('/content/drive')
auth.authenticate_user()
creds, _ = default()
service = build('drive', 'v3', credentials=creds)

print(f"Target ID: {TARGET_FOLDER_ID}")
print("Scanning start... (Sirf PDFs collect ho rahi hain)")

# Step 2: Scanning Logic
def scan_folder_recursive(folder_id):
    found_items = []
    folders_to_process = [folder_id]
    processed_count = 0

    while folders_to_process:
        current_id = folders_to_process.pop(0)

        page_token = None
        while True:
            try:
                query = f"'{current_id}' in parents and trashed = false and (mimeType = 'application/vnd.google-apps.folder' or mimeType = 'application/pdf')"

                results = service.files().list(
                    q=query,
                    fields="nextPageToken, files(id, name, mimeType)",
                    includeItemsFromAllDrives=True,
                    supportsAllDrives=True,
                    pageSize=1000,
                    pageToken=page_token
                ).execute()

                items = results.get('files', [])

                for item in items:
                    if item['mimeType'] == 'application/pdf':
                        found_items.append(item)
                    elif item['mimeType'] == 'application/vnd.google-apps.folder':
                        folders_to_process.append(item['id'])

                page_token = results.get('nextPageToken')
                if not page_token:
                    break
            except Exception as e:
                print(f"Skipping folder due to error: {e}")
                break

        processed_count += 1
        print(f"Folders Scanned: {processed_count} | PDFs Found: {len(found_items)}", end='\r')

    return found_items

# --- EXECUTION ---
raw_items = scan_folder_recursive(TARGET_FOLDER_ID)

print(f"\nScanning Complete! Total PDFs found: {len(raw_items)}")
print("Excel file taiyar kar raha hun...")

# Step 3: Data Cleaning & Excel Prep
data = []

for item in raw_items:
    original_name = item['name']
    file_id = item['id']

    # --- TITLE CLEANING LOGIC START (UPDATED) ---

    # 1. Sabse pehle .pdf extension hatao (Case insensitive)
    clean_title = re.sub(r'\.pdf$', '', original_name, flags=re.IGNORECASE)

    # 2. Specific website branding hatao
    clean_title = clean_title.replace('(www.urdunovelbanks.com)', '')

    # 3. NEW RULE: Replace +, -, _ with Space
    clean_title = re.sub(r'[+_\-]', ' ', clean_title)

    # 4. Extra spaces fix karo
    clean_title = re.sub(r'\s+', ' ', clean_title)

    # 5. Shuru aur aakhir ki spaces saaf karo
    clean_title = clean_title.strip()

    # --- TITLE CLEANING LOGIC END ---

    link = f"https://drive.google.com/file/d/{file_id}/view"

    data.append({
        "Titles": clean_title,
        "Links": link
    })

# Step 4: A-Z Sorting + Save to Excel
if data:
    # DataFrame banane se pehle ya baad mein sort kar sakte hain
    df = pd.DataFrame(data)

    # Titles column ko A to Z sort karo (case-insensitive ke liye str.lower use kiya)
    df = df.sort_values(by="Titles", key=lambda x: x.str.lower(), ignore_index=True)

    output_path = f"/content/drive/MyDrive/{OUTPUT_FILENAME}"
    df.to_excel(output_path, index=False)

    print(f"\nSUCCESS! Excel file ban gayi hai (Titles A-Z sorted).")
    print(f"Location: {output_path}")
    files.download(output_path)
else:
    print("\nKoi PDF nahi mili.")